# 네이버 뉴스 크롤러 (하드 코드용)

### 본 크롤러는 네이버 뉴스 크롤러 입니다.

## ========== ver 0.4 =========

#### - <u>세부 파라미터값이 있습니다. 함수에서 직접 조절해주세요.</u>

## ========== ver 0.3 =========

#### - naver.news 개편에 따라 크롤링 태그 변경이 있었습니다. (update by. daesun97)
#### - 전처리 기능을 더욱 강화했습니다. (update by. daesun97)
#### - 기사 카테고리 분류가 추가되었습니다.
#### - 기간내 특정 일자들의 기사를 놓치는 문제를 해결하고자 알고리즘 수정이 있었습니다.
#### - 자동으로 검색 일자를 변경해 최소 1주일 단위로 기사를 수집합니다.

## ========== ver 0.2 =========

#### - 본문의 전처리가 추가되었습니다. (update by. daesun97)

## ========== ver 0.1 =========
#### - 본 크롤러는 안내에 맞게 날짜를 입력해주셔야 합니다.
#### - 처음 사용하시는 분이라면 시작 전 본인의 컴퓨터에 !pip install fake_useragent 를 꼭 해주세요.
#### - fake_useragent로 ip차단걱정을 덜었습니다.
#### - 가급적 로컬 환경에서 사용해주세요. (colab 환경에서 실험을 아직 안해봤습니다)
#### - excel로 자동 저장됩니다.
#### - 각종 이슈 발생시 꼭 알려주세요.




###### (제작자 : suhykii)
###### 참조 블로그 리스트


In [ ]:
#!pip install fake_useragent

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import time
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from fake_useragent import UserAgent
import csv

In [28]:
####### 여기를 수정해주세요 ####### 

keyword = "네이버"
start_day = "2017-02-01"
last_day = "2022-03-31"

###################################


df = pd.DataFrame(columns=['title', 'category', 'date', 'paper', 'url', 'content'])

last_day = pd.to_datetime(last_day)
front_day = pd.to_datetime(start_day)
back_day = pd.to_datetime(start_day) + timedelta(days=6)
new_len=0
while (last_day >= back_day) & (last_day >= front_day):
    front_day = str(front_day)[:10]
    back_day = str(back_day)[:10]
    s_d_f = front_day.replace("-",".")
    e_d_f = back_day.replace("-",".")
    s_d_b = front_day.replace("-","")
    e_d_b = back_day.replace("-","")
    page=1
    less_news_cnt = 0
    old_len = new_len
    while less_news_cnt < 8:
        url = f"https://search.naver.com/search.naver?where=news&sm=tab_pge&query={keyword}&sort=0&photo=0&field=0&pd=3&ds={s_d_f}&de={e_d_f}&cluster_rank=47&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from{s_d_b}to{e_d_b},a:all&start="+str(page)
        
        response = requests.get(url)
        html = response.content
        soup = BeautifulSoup(html, 'html.parser')

        for urls in soup.select("a.info"):
            naver_news_cnt = len([urls for urls in soup.select("a.info") if urls["href"].startswith("https://news.naver.com") ])
            if naver_news_cnt < 3:
                less_news_cnt+=1
            
            try:
                if urls["href"].startswith("https://news.naver.com"):
                    news_detail = []

                    ua=UserAgent()
                    headers = {"User-Agent" : ua.random}

                    breq = requests.get(urls["href"], headers = headers)
                    bsoup = BeautifulSoup(breq.content, 'html.parser')

                    title = bsoup.select('h2.media_end_head_headline')[0].text
                    news_detail.append(title)

                    category = bsoup.select('.media_end_categorize_item')[0].text
                    news_detail.append(category)

                    date = bsoup.select('._ARTICLE_DATE_TIME')[0].text[:10]
                    date = date.replace(".","")
                    news_detail.append(date)

                    pcompany = bsoup.select('.media_end_linked_more_point')[0].text
                    news_detail.append(pcompany)

                    news_detail.append(urls["href"])

                    _text = bsoup.select('#newsct_article')[0].get_text().replace('\n', "")
                    btext = _text.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
                    btext=btext[:btext.find(('▶'))]
                    btext=btext[:btext.find(('▷'))]
                    btext=re.sub('[【<\[].*?[>\]】]', '', btext)
                    btext=re.sub('[◆©ⓒ■◇]', '', btext)
                    btext=re.sub('[a-zA-Z0-9+-_.]+@[a-zA-Z0-9]+\.[a-zA-Z0-9-.]+\.*[a-z]*','',btext)
                    btext=re.sub('[0-9]+\.[0-9]*\.[0-9]*','',btext)
                    btext=re.sub('동+영+상','',btext)
                    btext=re.sub('[뉴]+[스]','',btext)
                    btext=re.sub('사+진+\=','',btext)
                    btext=re.sub('\(+사+진+=+.*?\)','',btext)
                    btext=re.sub('\(+[서]+[울]+=+[연]+[합]+\)','',btext)
                    btext=re.sub('....[기]+[자]+.=','',btext)
                    btext=re.sub('...[기]+[자]','',btext)
                    btext=re.sub('연+합+뉴+스','',btext)
                    btext=re.sub('\(+\)+네+이+버+.*','',btext)
                    btext=re.sub('.+제+공+\=','',btext)

                    news_detail.append(btext.strip())

                    df.loc[len(df)] = news_detail
            except Exception as e:
                pass


        page += 10
    new_len = len(df)
    print("\n")
    print(f"{front_day}부터 {back_day}까지 기사를",new_len-old_len,"개 추가했습니다.")
    front_day = pd.to_datetime(front_day) + timedelta(days=7)
    back_day = pd.to_datetime(back_day) + timedelta(days=7)
    if back_day >= last_day:
        back_day = last_day
df.to_excel(f'{keyword}_ 전체 데이터 .xlsx',index=False,encoding='utf-8-sig')
print(f"현재 폴더에 {keyword}_ 전체 데이터 .xlsx 이름으로 저장했습니다.")

KeyboardInterrupt: 

In [27]:
df.to_excel(f'{keyword}_ 테스트 .xlsx',index=False,encoding='utf-8-sig')